Testing multifits loading and using `gettp` on multiple files.

In [1]:
from dysh.fits.gbtfitsload import GBTFITSLoad
import dysh.util as util

In [2]:
# Point it to the testdata directory on the root dysh directory.
testdata = util.get_project_testdata()
#testdata = "/home/sandboxes/psalas/Dysh/dysh/testdata"
# The folder with multiple SDFITS files.
data_path = testdata  / "AGBT18B_354_03/AGBT18B_354_03.raw.vegas"

In [3]:
sdf = GBTFITSLoad(data_path, verbose=False)

In [4]:
sdf.summary()

,SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATIO
0,6,W49N,7.0,OffOn,1,45.077148,44.979549,4,2,4,1,128.090129,50.305025
1,7,W49N,7.0,OffOn,2,45.077148,44.979549,4,2,4,1,129.920889,50.435658


In [5]:
sdf._index.columns

Index(['OBJECT', 'BANDWID', 'DATE-OBS', 'DURATION', 'EXPOSURE', 'TSYS',
       'TDIM7', 'TUNIT7', 'CTYPE1', 'CRVAL1', 'CRPIX1', 'CDELT1', 'CTYPE2',
       'CRVAL2', 'CTYPE3', 'CRVAL3', 'CRVAL4', 'OBSERVER', 'OBSID', 'SCAN',
       'OBSMODE', 'FRONTEND', 'TCAL', 'VELDEF', 'VFRAME', 'RVSYS', 'OBSFREQ',
       'LST', 'AZIMUTH', 'ELEVATIO', 'TAMBIENT', 'PRESSURE', 'HUMIDITY',
       'RESTFREQ', 'FREQRES', 'EQUINOX', 'RADESYS', 'TRGTLONG', 'TRGTLAT',
       'SAMPLER', 'FEED', 'SRFEED', 'FEEDXOFF', 'FEEDEOFF', 'SUBREF_STATE',
       'SIDEBAND', 'PROCSEQN', 'PROCSIZE', 'PROCSCAN', 'PROCTYPE', 'LASTON',
       'LASTOFF', 'TIMESTAMP', 'QD_XEL', 'QD_EL', 'QD_BAD', 'QD_METHOD',
       'VELOCITY', 'ZEROCHAN', 'DOPFREQ', 'ADCSAMPF', 'VSPDELT', 'VSPRVAL',
       'VSPRPIX', 'SIG', 'CAL', 'CALTYPE', 'TWARM', 'TCOLD', 'CALPOSITION',
       'IFNUM', 'PLNUM', 'FDNUM', 'HDU', 'BINTABLE', 'ROW', 'PROC', '_OBSTYPE',
       '_SUBOBSMODE'],
      dtype='object')

In [6]:
#sdf._index['TRGTLONG'],sdf._index['CRVAL2']
sdf._index['VELOCITY']
sdf._index['CTYPE1']

0      FREQ-OBS
1      FREQ-OBS
2      FREQ-OBS
3      FREQ-OBS
4      FREQ-OBS
         ...   
123    FREQ-OBS
124    FREQ-OBS
125    FREQ-OBS
126    FREQ-OBS
127    FREQ-OBS
Name: CTYPE1, Length: 128, dtype: object

In [7]:
from astropy.wcs import WCS
header = sdf._index.iloc[0].dropna().to_dict()
w=WCS(header=header)

In [ ]:
set(["VELOCITY","CRVAL1", "FOO"]) <= header.keys()

Up to this point, it works fine. It tells you there are multiple IFs (spectral windows).

Now, when we try to calibrate something, it breaks down.

In [ ]:
ps_scans = sdf.getps(6, ifnum=2)

In [ ]:
ps_scans[0].timeaverage().plot()

In [ ]:
tg = GBTFITSLoad(data_path / "AGBT18B_354_03.raw.vegas.B.fits")
tg.summary()

In [ ]:
tg.onoff_scan_list([6,7],ifnum=1)

# Minimal failing example

In [ ]:
from dysh.fits.gbtfitsload import GBTFITSLoad
import dysh.util as util
 
 # Point it to the testdata directory on the root dysh directory.
testdata = util.get_project_testdata()
data_path = testdata  / "AGBT18B_354_03.raw.vegas"

sdf = GBTFITSLoad(data_path)
sdf.udata("IFNUM")
 
for ifnum in sdf.udata("IFNUM"):
    try:
        print(f"trying {ifnum}")
        ps_scan = sdf.getps([6], ifnum=ifnum)
    except Exception as e:
        print(f"Exception raised for ifnum={ifnum}")
        print(e)


In [ ]:
ps_scan = sdf.getps([6], ifnum=3)
ps_scan

In [ ]:
sdf = GBTFITSLoad(data_path)

In [ ]:
for s in sdf._sdf:
    print(s.filename)

In [ ]:
sdf.summary()


In [ ]:
fA="/home/mpound/src/dysh/testdata/AGBT18B_354_03.raw.vegas/AGBT18B_354_03.raw.vegas.A.fits"
sA = GBTFITSLoad(fA)

In [ ]:
sA.summary(verbose=True)

In [ ]:
fD="/home/mpound/src/dysh/testdata/AGBT18B_354_03.raw.vegas/AGBT18B_354_03.raw.vegas.D.fits"
sD = GBTFITSLoad(fD)

In [ ]:
sD.summary()

In [ ]:
sD.getps(scans=7,ifnum=3)

In [ ]:
sD.index(bintable=None,fitsindex=0)

In [ ]:
sA.getps(scans=7,ifnum=2)